# EfficientDet Pytorch Inference Starter

Hi Again Everyone,

First of all - Go [here](https://www.kaggle.com/mikeb127/efficientdet-pytorch-training-starter) to get training starter. Submitting this notebook as is without properly training the starter will probably result in a pretty low score as the starter has only been trained for an epoch or two. You will also have to add your classifier model.

First of all, lets load all our dependencies:

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

Next, lets just get the imports:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)  
import torch
import torch.nn.functional as F
import torchvision
import sys
sys.path.insert(0, "../input/timmefficienctdetpytorchstable/archive")
sys.path.insert(0, "../input/omegaconf")
import traceback
from tqdm import tqdm
from torch.utils.data import Dataset
import torchvision.models as models
from PIL import Image
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut #Apply a VOI lookup table or windowing operation to arr.
from torch import optim
from effdet import *
from effdet.efficientdet import HeadNet
from effdet.anchors import Anchors, AnchorLabeler, generate_detections, MAX_DETECTION_POINTS
from effdet.loss import DetectionLoss

import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test/'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))

Normalizer = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])

Ok, below is all the code I had to modify to get things to work. This is not a tidy solution but works for now. If there is interest I will incorporate these changes into the referenced package so it can be used going forward

In [ ]:
def _post_process(config, cls_outputs, box_outputs):
    """Selects top-k predictions.

    Post-proc code adapted from Tensorflow version at: https://github.com/google/automl/tree/master/efficientdet
    and optimized for PyTorch.

    Args:
        config: a parameter dictionary that includes `min_level`, `max_level`,  `batch_size`, and `num_classes`.

        cls_outputs: an OrderDict with keys representing levels and values
            representing logits in [batch_size, height, width, num_anchors].

        box_outputs: an OrderDict with keys representing levels and values
            representing box regression targets in [batch_size, height, width, num_anchors * 4].
    """
    batch_size = cls_outputs[0].shape[0]
    cls_outputs_all = torch.cat([
        cls_outputs[level].permute(0, 2, 3, 1).reshape([batch_size, -1, config.num_classes])
        for level in range(config.num_levels)], 1)

    box_outputs_all = torch.cat([
        box_outputs[level].permute(0, 2, 3, 1).reshape([batch_size, -1, 4])
        for level in range(config.num_levels)], 1)

    _, cls_topk_indices_all = torch.topk(cls_outputs_all.reshape(batch_size, -1), dim=1, k=MAX_DETECTION_POINTS)
    indices_all = cls_topk_indices_all / config.num_classes
    classes_all = cls_topk_indices_all % config.num_classes
    
    indices_all = indices_all.type(torch.long)

    box_outputs_all_after_topk = torch.gather(
        box_outputs_all, 1, indices_all.unsqueeze(2).expand(-1, -1, 4))

    cls_outputs_all_after_topk = torch.gather(
        cls_outputs_all, 1, indices_all.unsqueeze(2).expand(-1, -1, config.num_classes))
    cls_outputs_all_after_topk = torch.gather(
        cls_outputs_all_after_topk, 2, classes_all.unsqueeze(2))

    return cls_outputs_all_after_topk, box_outputs_all_after_topk, indices_all, classes_all


class DetBenchEvalMB(torch.nn.Module):
    
    def __init__(self, model, config, device):
        super(DetBenchEvalMB, self).__init__()
        self.config = config
        self.model = model
        self.anchors = Anchors(
            config.min_level, config.max_level,
            config.num_scales, config.aspect_ratios,
            config.anchor_scale, config.image_size,device)

    def forward(self, x, image_scales):
        class_out, box_out = self.model(x)
        class_out, box_out, indices, classes = _post_process(self.config, class_out, box_out)

        batch_detections = []
        for i in range(x.shape[0]):
            detections = generate_detections(
                class_out[i], box_out[i], self.anchors.boxes, indices[i], classes[i], image_scales[i])
            batch_detections.append(detections)
        return torch.stack(batch_detections, dim=0)

        
def get_efficientDet():
    
    config = get_efficientdet_config('tf_efficientdet_d1')
    net = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    return net, config

model, config = get_efficientDet()

        
class out_class(torch.nn.Module):
    
    def __init__(self,backbone):
        super(out_class, self).__init__()
        self.backbone = backbone
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self,x):        
        x = self.backbone(x)
        x = self.sigmoid(x)
        return x
        
        
def post_process_outputs(output, width, height):
    for z in range(0,len(output)):
        #Returns Values as xywh. Sort this out
        output[z][:,0] = output[z][:,0] * width[z]/512
        output[z][:,1] = output[z][:,1] * height[z]/512 
        output[z][:,2] = output[z][:,2] * width[z]/512 
        output[z][:,3] = output[z][:,3] * height[z]/512
        output[z,:,3] = output[z,:,3] + output[z,:,1]
        output[z,:,2] = output[z,:,2] + output[z,:,0]
    return output

Let's do our dataloader. Thanks here goes to users tensorchoko and raddar for the DICOM processing methods

https://www.kaggle.com/tensorchoko/siim-yolov5-predict

https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way




In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    # I pinched it from https://www.kaggle.com/tensorchoko/siim-yolov5-predict
    dicom = pydicom.dcmread(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data


BATCH_SIZE = 2

def collate_fn(batch):
    return tuple(zip(*batch))


class COVIDXRay_Dataset(Dataset):

    def __init__(self, dcm_file_list, transform):

        self.dcm_file_list = dcm_file_list
        self.transform = transform

    def __len__(self):
        
        return len(self.dcm_file_list)

    def __getitem__(self, idx):
        
        #First of all, we load the image
        image = read_xray(self.dcm_file_list[idx])
        img = Image.fromarray(image).convert('RGB')
        width = img.size[0]
        height = img.size[1]
        img = img.resize((512,512))
        img = self.transform(img)
        
        #Lets get our desired output at the study level
        file = self.dcm_file_list[idx].split('/')
        
        #First of all we get our study level ground truth
        study_file = file[5] + '_study'
        
        #Now our image level ground truth
        image_file = file[7].split('.')[0] + '_image'
        dummy = torch.Tensor(np.ones(1))
        return img, image_file, study_file, width, height, dummy
    
    
train_dataset = COVIDXRay_Dataset(
        files, Normalizer
)

data_loader_test = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=1,
    shuffle=False,
    collate_fn=collate_fn
)

Finally update the code below to load the model we trained previously and create a submission file

In [ ]:
device = torch.device("cuda:0")
#@TODO: Replace this with the link to your copy of the training notebook!!
a = torch.load(###Your notebook here###)
model.load_state_dict(a)
model = model.to(device)
model = DetBenchEvalMB(model,config,device)
model.eval()

tk0 = tqdm(data_loader_test, desc="Iteration")
box_detections = []

for batch_idx, (input, image_file, study_file, width, height,dummy) in enumerate(tk0):
    input = [i.to(device) for i in input]
    dummy = [d.to(device) for d in dummy]
    input = torch.stack(input)
    output = model(input,dummy)
    output = post_process_outputs(output,width,height)
    output = output.cpu().detach().numpy()
    for q in range(0,output.shape[0]):
        boxes_string = ''
        for b in range(0,output.shape[1]):
            if output[q,b,4] > 0.1:          
                boxes_string = boxes_string + 'opacity ' + str output[q,b,4]) + ' ' + str(int(output[q,b,0])) + ' ' + str(int(output[q,b,1])) + ' ' + str(int(output[q,b,2])) + ' ' + str(int(output[q,b,3])) + ' '
        if len(boxes_string) == 0:
            boxes_string = 'none 1 0 0 1 1 '
        boxes_string = boxes_string[:len(boxes_string)-1]
        box_string_rec = [image_file[q], boxes_string]
        box_detections.append(box_string_rec)
  
df_box_detections = pd.DataFrame(box_detections,columns=['id', 'PredictionString'])
df_box_detections.to_csv('/kaggle/working/submission.csv',index = False)

And that should be it! You can now combine this with your classification model.

Thanks for reading